In [ ]:
# 这是一个命令行的小游戏，宝藏在最右边 T 寻找到宝藏就是胜利

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
N_STATES = 6 # the length of the 1 dimensional world
ACTIONS = ['left', 'right'] # available actions 只能左右移动
EPSILON = 0.9   # greedy police 0.9 的可能选择最大值
ALPHA = 0.1 # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 13   # maximum episodes 最多循环次数
FRESH_TIME = 0.3    # fresh time for one move

In [4]:
def build_q_tabel(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),
        columns=actions
    )
    return table

In [5]:
# return right or left
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    # act non-greedy or state-action have no value
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):
        action_name = np.random.choice(ACTIONS)
    # act greedy
    else:
        # replace argmax to idxmax as argmax means a different function in newer version of pandas
        action_name =state_actions.idxmax()
    return action_name

In [6]:
# 如果选对了给一个评分 1 选错了给评分 0 终点退出
def get_env_feedback(S, A):
    if A == 'right':
        if S == N_STATES - 2:
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:
        R = 0
        # reach the wall
        if S == 0:
            S_ = 0
        else:
            S_ = S - 1
    return S_, R

In [7]:
def update_env(S, episode, step_counter):
    env_list = ['-']*(N_STATES - 1) + ['T']
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

$$
q_t(s_t,a_t) = q_{t-1}(s_{t-1}, a_{t-1})+\alpha (r_t + \gamma * max_aq_{t-1}{(s_t)}-q_{t-1}(s_{t-1},a_{t-1}))
$$

In [8]:
def rl():
    q_table = build_q_tabel(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()
            else:
                q_target = R
                is_terminated = True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)
            S = S_
            
            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table

In [9]:
q_table = rl()
print('\r\nQ-table:\n')
print(q_table)

                                
Q-table:

       left     right
0  0.000027  0.004261
1  0.000000  0.024929
2  0.000007  0.108445
3  0.000000  0.340790
4  0.000000  0.745813
5  0.000000  0.000000
